In [ ]:
%%writefile submission.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, translate, adjacent_positions
import numpy as np
import math

last_action = None

def agent(obs_dict, config_dict):
    global last_action

    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    
    search_len = min([configuration.columns, configuration.rows]) // 2
    print("search_len", search_len)
    
    field = np.zeros([configuration.rows, configuration.columns])
    
    goose_reward = -1
    for goose in observation.geese:
        body_len = len(goose)
        for i, g in enumerate(goose):
            field[row_col(g, configuration.columns)] = (body_len - i) * goose_reward

    opponents = [
        goose
        for index, goose in enumerate(observation.geese)
        if index != observation.index and len(goose) > 0
    ]
    for opponent in opponents:
        opponent_reward = math.ceil(len(opponent) / 3) * goose_reward
        for opponent_head_adjacent in adjacent_positions(opponent[0], configuration.columns, configuration.rows):
            if field[row_col(opponent_head_adjacent, configuration.columns)] >= 0:
                field[row_col(opponent_head_adjacent, configuration.columns)] = opponent_reward
    
    free = np.count_nonzero(field >= 0)
    survive = len([
        goose
        for index, goose in enumerate(observation.geese)
        if len(goose) > 0
    ])
    food_reward = free // survive
    for food in observation.food:
        field[row_col(food, configuration.columns)] = field[row_col(food, configuration.columns)] + food_reward

    print(field)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)

    north_reward = 0
    if last_action == Action.SOUTH.name:
        north_reward = -100000
    else:
        now_player_head = player_head
        for i in range(search_len):
            now_player_head = translate(now_player_head, Action.NORTH, configuration.columns, configuration.rows)
            if i == 0:
                if field[row_col(now_player_head, configuration.columns)] < 0:
                    north_reward = -10000
                    break
            north_reward = north_reward + (field[row_col(now_player_head, configuration.columns)] * (search_len - i))
            remain_len = search_len - (i + 1)
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.EAST, configuration.columns, configuration.rows)
                north_reward = north_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.WEST, configuration.columns, configuration.rows)
                north_reward = north_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
    print("north_reward", north_reward)
            
    south_reward = 0
    if last_action == Action.NORTH.name:
        south_reward = -100000
    else:
        now_player_head = player_head
        for i in range(search_len):
            now_player_head = translate(now_player_head, Action.SOUTH, configuration.columns, configuration.rows)
            if i == 0:
                if field[row_col(now_player_head, configuration.columns)] < 0:
                    south_reward = -10000
                    break
            south_reward = south_reward + (field[row_col(now_player_head, configuration.columns)] * (search_len - i))
            remain_len = search_len - (i + 1)
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.EAST, configuration.columns, configuration.rows)
                south_reward = south_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.WEST, configuration.columns, configuration.rows)
                south_reward = south_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
    print("south_reward", south_reward)

    east_reward = 0
    if last_action == Action.WEST.name:
        east_reward = -100000
    else:
        now_player_head = player_head
        for i in range(search_len):
            now_player_head = translate(now_player_head, Action.EAST, configuration.columns, configuration.rows)
            if i == 0:
                if field[row_col(now_player_head, configuration.columns)] < 0:
                    east_reward = -10000
                    break
            east_reward = east_reward + (field[row_col(now_player_head, configuration.columns)] * (search_len - i))
            remain_len = search_len - (i + 1)
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.NORTH, configuration.columns, configuration.rows)
                east_reward = east_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.SOUTH, configuration.columns, configuration.rows)
                east_reward = east_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
    print("east_reward", east_reward)

    west_reward = 0
    if last_action == Action.EAST.name:
        west_reward = -100000
    else:
        now_player_head = player_head
        for i in range(search_len):
            now_player_head = translate(now_player_head, Action.WEST, configuration.columns, configuration.rows)
            if i == 0:
                if field[row_col(now_player_head, configuration.columns)] < 0:
                    west_reward = -10000
                    break
            west_reward = west_reward + (field[row_col(now_player_head, configuration.columns)] * (search_len - i))
            remain_len = search_len - (i + 1)
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.NORTH, configuration.columns, configuration.rows)
                west_reward = west_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
            temp_player_head = now_player_head
            for j in range(remain_len):
                temp_player_head = translate(temp_player_head, Action.SOUTH, configuration.columns, configuration.rows)
                west_reward = west_reward + (field[row_col(temp_player_head, configuration.columns)] * (search_len - i - j - 1))
    print("west_reward", west_reward)

    max_reward = max([north_reward,south_reward,east_reward,west_reward])        
    
    if north_reward == max_reward:
        print('NORTH')
        last_action = Action.NORTH.name
        return Action.NORTH.name
    if south_reward == max_reward:
        print('SOUTH')
        last_action = Action.SOUTH.name
        return Action.SOUTH.name
    if east_reward == max_reward:
        print('EAST')
        last_action = Action.EAST.name
        return Action.EAST.name
    print('WEST')
    last_action = Action.WEST.name
    return Action.WEST.name

In [ ]:
from kaggle_environments import *

env = make("hungry_geese", configuration={"columns": 11, "rows": 7,  "hunger_rate": 40, "min_food": 2}, debug=False)
output = env.run(['submission.py', 'random', 'greedy', 'greedy'])
env.render(mode='ipython', width=800, height=700)